In [ ]:
%reset -f

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open("/content/names.txt",'r').read().splitlines()

In [ ]:
'''
words = words[:5]
words
'''

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [ ]:
'''
b = {}
for w in words:
  chs = ['<S>'] +list(w) + ['<E>']
  for ch1,ch2 in zip(chs,chs[1:]):
    bigram = (ch1,ch2)
    b[bigram] = b.get(bigram,0) + 1

sorted(b.items(), key = lambda kv: -kv[1])
'''

In [3]:

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}


In [5]:
N = torch.zeros((27,27),dtype=torch.int32)

In [6]:
# UPDATE MATRIX WITH COUNTS
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs,chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1,ix2] += 1

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(N, cmap='Blues')

for i in range(12):
  for j in range(12):
    chstr = itos[i] + itos[j]
    plt.text(j,i,chstr, ha="center", va="bottom", color="gray")
    plt.text(j,i, N[i,j].item(), ha="center", va="top",color="gray")


In [7]:
# GENERATING SAMPLES
g = torch.Generator().manual_seed(2147983647)

P = (N+1).float()
P /= P.sum(1, keepdim=True)

for i in range(3):
  out = []
  ix = 0
  while True:
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1,replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

ta.
arovid.
ya.


In [8]:
# NEGATIVE LOGLIKELIHOOD
log_likelihood = 0.0
n = 0

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs,chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1,ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-559951.5625)
nll=tensor(559951.5625)
2.4543561935424805


In [9]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27,27), generator = g, requires_grad=True)

In [10]:
xs, ys = [], []

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs,chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

num = xs.nelement()
print('Number of examples: ', num)

Number of examples:  228146


In [ ]:
'''
X = F.one_hot(xs, num_classes = 12).float()
logits = X @ W
counts = logits.exp()
probs = counts/ counts.sum(1, keepdims=True)
'''

In [ ]:
'''
probs[torch.arange(2),[1,2]]
'''

In [ ]:
'''
probs
'''

In [13]:
'''
probs.shape
'''

torch.Size([228146, 27])

In [22]:
for k in range(1):

  X = F.one_hot(xs, num_classes = 27).float()
  logits = X @ W
  counts = logits.exp()
  probs = counts/ counts.sum(1, keepdims=True)
  loss = -probs[torch.arange(228146),ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())

  W.grad = None
  loss.backward()

  W.data += -50*W.grad

3.3788065910339355


In [26]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):

  out = []
  ix = 0
  while True:

    X = F.one_hot(torch.tensor([ix]), num_classes = 27).float()
    logits = X @ W
    counts = logits.exp()
    p = counts/counts.sum(1, keepdims=True)

    ix = torch.multinomial(p, num_samples = 1, replacement = True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

juwjdjdjanaqydjufhfbywecn.
a.
jiin.
toziasz.
twt.
